In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn as nn
import torch.optim as optim

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Mounting the drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import tarfile

Extracting the files and folders

In [ ]:
file = tarfile.open('/content/drive/MyDrive/Augnito_Assignment/UrbanSound8K.tar.gz')
file.extractall('./')
file.close()

In [ ]:
df = pd.read_csv("/content/UrbanSound8K/metadata/UrbanSound8K.csv")
df.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [ ]:
df["path"] = "/content/UrbanSound8K/audio/fold"+df['fold'].astype(str)+"/"+df['slice_file_name'].astype(str)

In [ ]:
df = df[['path','classID','class']]
df.head()

,path,classID,class
0,/content/UrbanSound8K/audio/fold5/100032-3-0-0...,3,dog_bark
1,/content/UrbanSound8K/audio/fold5/100263-2-0-1...,2,children_playing
2,/content/UrbanSound8K/audio/fold5/100263-2-0-1...,2,children_playing
3,/content/UrbanSound8K/audio/fold5/100263-2-0-1...,2,children_playing
4,/content/UrbanSound8K/audio/fold5/100263-2-0-1...,2,children_playing


In [ ]:
import torchaudio
import torchaudio.transforms as T
import torch.functional as F
import random
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
import librosa

# **Zero shot**

In [ ]:
!pip install openl3

  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.4/323.4 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openl3: filename=openl3-0.4.2-py2.py3-none-any.whl size=249327032 sha256=70f695ae3f207f91b87f51f5f1f2e000a9e8af7d987bf5ad74ecc520ef2e5e37
  Stored in directory: /root/.cache/pip/wheels/d0/4d/0a/e57b1dc8ead91b3c5709d9de4f02d1cdd3a91f609a8f1c1062
  Created wheel for kapre: filename=kapre-0.3.7-py3-none-any.whl size=29603 sha256=b2d964d42381c3a4254da75acf19e0923ad223c8602b3765378332e188ed3124
  Stored in directory: /root/.cache/pip/wheels/3e/54/f9/37e9f36590a5431dc207f7dc0eb2e607ff8958d5728e45ef58
  Created wheel for resampy: filename=resampy-0.2.2-py3-none-any.whl size=320706 sha256=c0184c90d9d6d07b79c4074b222a9bfab49074326802f1ec746286c0dea6f1ad
  Stored in directory: /root

In [ ]:
import openl3
import soundfile as sf
from sklearn.svm import SVC

In [ ]:
path = df.iloc[1]['path']
# print(path)
class_id = df.iloc[1]['classID']
# spectrogram = pre_process(path,44100,4000)

In [ ]:
def embeddings(path):
  signal,sr = sf.read(path)
  embeddings, _ = openl3.get_audio_embedding(signal,sr)
  embeddings = torch.tensor(embeddings).to(device)
  return embeddings


In [ ]:
random_df = df.sample(frac=0.05)

In [ ]:
audio_paths = list(random_df['path'])
labels = list(random_df['class'])

In [ ]:
len(labels)

437

In [ ]:
classes = list(set(labels))
len(classes)

10

Pre-processing

In [ ]:
import librosa
def pre_process1(path,new_sr,max_ms):
  signal, sr = librosa.load(path,sr=new_sr)

  # print(signal)
  # print(sr)
  # if(signal.shape[0]==1):
    # signal = torch.cat([signal,signal])

  # new_sr = 48000
  # re_signal1 = T.Resample(sr,new_sr)(signal[:1,:])
  # re_signal2 = T.Resample(sr,new_sr)(signal[1:,:])
  # re_signal1 = torch.cat([re_signal1,re_signal2])


  max_len = new_sr//1000 * max_ms
  # print(max_len,re_signal1.shape[1])

  if(signal.shape[0]<max_len):
    signal = np.pad(signal,int(np.ceil((max_len-signal.shape[0])/2)),mode='reflect')
  else:
    signal = signal[:max_len]

  embeddings, _ = openl3.get_audio_embedding(signal,sr)
  embeddings = torch.tensor(embeddings).to(device)
  return embeddings







In [ ]:
X = []
for i,a in enumerate(audio_paths):
  if(i%100==0):
    print(i)
  X.append(pre_process1(a,44100,4000).flatten().cpu().numpy())




0
2/2 [==============================] - 1s 148ms/step
100
2/2 [==============================] - 0s 125ms/step
200
2/2 [==============================] - 1s 123ms/step
300
2/2 [==============================] - 1s 127ms/step
400
2/2 [==============================] - 1s 130ms/step


In [ ]:
classes

['air_conditioner',
 'children_playing',
 'engine_idling',
 'street_music',
 'jackhammer',
 'drilling',
 'gun_shot',
 'siren',
 'dog_bark',
 'car_horn']

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,classification_report

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=15)

In [ ]:
classifier = make_pipeline(StandardScaler(), SVC(kernel='linear'))
classifier.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(kernel='linear'))])

Evaluation

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 81.82%


In [ ]:
precision = precision_score(y_test, y_pred,average = "weighted")
print(f"Precision: {precision:.2f}")

Precision: 0.83


In [ ]:
recall = recall_score(y_test, y_pred,average = "weighted")
print(f"Recall: {recall:.2f}")

Recall: 0.82


In [ ]:
f1score = f1_score(y_test, y_pred,average = "weighted")
print(f"F1_score: {f1score:.2f}")

F1_score: 0.81


In [ ]:
print(classification_report(y_test, y_pred))

                  precision    recall  f1-score   support

 air_conditioner       0.74      1.00      0.85        14
        car_horn       1.00      0.71      0.83         7
children_playing       1.00      1.00      1.00         2
        dog_bark       0.80      0.67      0.73         6
        drilling       0.90      0.82      0.86        11
   engine_idling       0.75      0.38      0.50         8
        gun_shot       0.75      0.75      0.75         4
      jackhammer       0.92      0.85      0.88        13
           siren       0.77      0.83      0.80        12
    street_music       0.79      1.00      0.88        11

        accuracy                           0.82        88
       macro avg       0.84      0.80      0.81        88
    weighted avg       0.83      0.82      0.81        88



# **Fine Tuning**

Pre-processing

In [ ]:
def spec_image(x):
  mean = x.mean()
  std = x.std()
  x_norm = (x-mean)/(std+1e-6)
  x_min,x_max = x_norm.min(),x_norm.max()
  x_scaled = 255*(x_norm-x_min)/(x_max-x_min)
  x_scaled = x_scaled.astype(np.uint8)
  return x_scaled

In [ ]:
def pre_process(path,new_sr,max_ms):
  signal, sr = librosa.load(path,sr=new_sr)

  # print(signal)
  # print(sr)
  # if(signal.shape[0]==1):
    # signal = torch.cat([signal,signal])

  # new_sr = 48000
  # re_signal1 = T.Resample(sr,new_sr)(signal[:1,:])
  # re_signal2 = T.Resample(sr,new_sr)(signal[1:,:])
  # re_signal1 = torch.cat([re_signal1,re_signal2])


  max_len = new_sr//1000 * max_ms
  # print(max_len,re_signal1.shape[1])

  if(signal.shape[0]<max_len):
    signal = np.pad(signal,int(np.ceil((max_len-signal.shape[0])/2)),mode='reflect')
  else:
    signal = signal[:max_len]

  spectrogram = librosa.feature.melspectrogram(y=signal,sr=sr,n_fft=1024,hop_length=None,n_mels=64)
  spec_db = librosa.power_to_db(spectrogram,top_db=80)
  spec_imaged = spec_image(spec_db)

  return spec_imaged[np.newaxis,...]

In [ ]:
class Sound_ds(Dataset):
  def __init__(self,df):
    self.df = df

  def __len__(self):
    return len(self.df)

  def __getitem__(self, inx):
    path = self.df.iloc[inx]['path']
    class_id = self.df.iloc[inx]['classID']
    spectrogram = pre_process(path,44100,4000)

    return spectrogram,class_id

In [ ]:
train_df,test_df = train_test_split(df,test_size=0.2,random_state=15)
train_df,val_df = train_test_split(train_df,test_size=0.1,random_state=15)

In [ ]:
train_dataset = Sound_ds(train_df)
val_dataset = Sound_ds(val_df)
test_dataset = Sound_ds(test_df)

In [ ]:
batch_size = 16
train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=batch_size,shuffle=False)
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

ResNet50 Model

In [ ]:
import torchvision.models as models

resnet_model = models.resnet50(pretrained=True)
resnet_model.fc = torch.nn.Linear(resnet_model.fc.in_features,10)
resnet_model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet_model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 67.3MB/s]


ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
lr = 2e-4
optimizer = optim.Adam(resnet_model.parameters(),lr=lr)
epochs = 50
loss_f = nn.CrossEntropyLoss()

Training

In [ ]:
train_loss = []
vali_loss = []

def train(model,loss_f,train_loader,val_loader,optimizer,train_loss,val_loss):
  for epoch in range(10):
    model.train()
    batch_losses = []
    for i,data in enumerate(train_loader):
      optimizer.zero_grad()
      x,y=data[0].to(device,dtype=torch.float32),data[1].to(device)

      y_pred = model(x)
      loss = loss_f(y_pred,y)
      loss.backward()
      batch_losses.append(loss.item())
      optimizer.step()

    train_loss.append(batch_losses)
    print(f'Epoch - {epoch} Train-Loss : {np.mean(train_loss[-1])}')

    model.eval()

    val_batch_losses = []

    with torch.no_grad():
      for i,data in enumerate(val_loader):
        x,y=data[0].to(device,dtype=torch.float32),data[1].to(device)
        y_pred = model(x)
        val_loss = loss_f(y_pred,y)
        val_batch_losses.append(val_loss.item())

    vali_loss.append(val_batch_losses)
    print(f'Epoch - {epoch} Validation-Loss : {np.mean(vali_loss[-1])}')




In [ ]:
train(resnet_model,loss_f,train_loader,val_loader,optimizer,train_loss,vali_loss)

Epoch - 0 Train-Loss : 0.9289162204465793
Epoch - 0 Validation-Loss : 0.6848569742657922
Epoch - 1 Train-Loss : 0.4961916004655925
Epoch - 1 Validation-Loss : 0.4256354915824803
Epoch - 2 Train-Loss : 0.3568891938491405
Epoch - 2 Validation-Loss : 0.3265262983570045
Epoch - 3 Train-Loss : 0.2612209692289318
Epoch - 3 Validation-Loss : 0.4936152378266508
Epoch - 4 Train-Loss : 0.21343236342404256
Epoch - 4 Validation-Loss : 0.6765704530883919
Epoch - 5 Train-Loss : 0.17061075515945562
Epoch - 5 Validation-Loss : 0.23573328098493881
Epoch - 6 Train-Loss : 0.14364029314255214
Epoch - 6 Validation-Loss : 0.39400295955552295
Epoch - 7 Train-Loss : 0.13471476723168652
Epoch - 7 Validation-Loss : 0.2495168896857649
Epoch - 8 Train-Loss : 0.12936198081004815
Epoch - 8 Validation-Loss : 0.24412437513555316
Epoch - 9 Train-Loss : 0.10663614732286782
Epoch - 9 Validation-Loss : 0.38381535752507095


Evaluating

In [ ]:
resnet_model.eval()

predictions = []
true_labels = []
correct_pred = 0
total_pred = 0
with torch.no_grad():
  for i,data in enumerate(test_loader):
    if(i%100==0):
      print(i)
    x,y=data[0].to(device,dtype=torch.float32),data[1].to(device)
    # print(x.shape)
    # print(y.shape)


    outputs = resnet_model(x)
    _,y_pred = torch.max(outputs,1)
    # print(y_pred,y)
    predictions.extend(y_pred.cpu().numpy())
    true_labels.extend(y.cpu().numpy())
    # correct_pred +=(y_pred==y).sum().item()
    # total_pred += y_pred.shape[0]



0
100


In [ ]:
accuracy = accuracy_score(predictions,true_labels)
print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 90.44%


In [ ]:
precision = precision_score(predictions,true_labels,average = "weighted")
print(f"Precision: {precision:.2f}")

Precision: 0.92


In [ ]:
recall = recall_score(predictions,true_labels,average = "weighted")
print(f"Recall: {recall:.2f}")

Recall: 0.90


In [ ]:
f1score = f1_score(predictions,true_labels,average = "weighted")
print(f"F1_score: {f1score:.2f}")

F1_score: 0.91


In [ ]:
print(classification_report(predictions,true_labels,target_names=sorted(classes)))

                  precision    recall  f1-score   support

 air_conditioner       0.96      0.88      0.92       219
        car_horn       0.88      0.95      0.91        83
children_playing       0.94      0.73      0.82       250
        dog_bark       0.91      0.96      0.94       177
        drilling       0.94      0.98      0.96       199
   engine_idling       0.95      0.96      0.95       201
        gun_shot       0.96      1.00      0.98        72
      jackhammer       0.99      0.85      0.92       218
           siren       0.98      0.94      0.96       205
    street_music       0.56      0.96      0.71       123

        accuracy                           0.90      1747
       macro avg       0.91      0.92      0.91      1747
    weighted avg       0.92      0.90      0.91      1747



Whisper Model

In [ ]:
!pip install datasets

In [ ]:
from transformers import WhisperModel,WhisperFeatureExtractor,AdamW
import datasets
from datasets import load_dataset, DatasetDict,  Audio

In [ ]:
df1 = df.sample(n=1500, random_state=15)
df1.head()

,path,classID,class
8443,/content/UrbanSound8K/audio/fold10/93567-8-0-1...,8,siren
1705,/content/UrbanSound8K/audio/fold4/144007-5-1-3...,5,engine_idling
3689,/content/UrbanSound8K/audio/fold5/17578-5-0-32...,5,engine_idling
4413,/content/UrbanSound8K/audio/fold10/181624-4-8-...,4,drilling
3597,/content/UrbanSound8K/audio/fold7/174284-6-2-0...,6,gun_shot


In [ ]:
train_df,test_df = train_test_split(df1,test_size=0.2,random_state=15)
train_df,val_df = train_test_split(train_df,test_size=0.1,random_state=15)

Preparing the data

In [ ]:
train_audio_dataset = datasets.Dataset.from_dict({"audio": train_df['path'].tolist(),"labels": train_df["classID"].tolist()}).cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
test_audio_dataset = datasets.Dataset.from_dict({"audio": test_df['path'].tolist(),"labels": test_df["classID"].tolist()}).cast_column("audio", Audio(sampling_rate=16_000))

val_audio_dataset = datasets.Dataset.from_dict({"audio": val_df['path'].tolist(),"labels": val_df["classID"].tolist()}).cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-base")
model = WhisperModel.from_pretrained("openai/whisper-base")

In [ ]:
class Sound_ds1(Dataset):
  def __init__(self,data):
    self.data = data

  def __len__(self):
    return len(self.data)

  def __getitem__(self, inx):
    signal = feature_extractor(self.data[inx]['audio']['array'],return_tensors="pt",sampling_rate=self.data[inx]['audio']['sampling_rate']).input_features
    signal_ids = torch.tensor([[1,1]])*model.config.decoder_start_token_id
    labels = np.array(self.data[inx]['labels'])

    return signal,signal_ids,torch.tensor(labels)


In [ ]:
train_dataset1 = Sound_ds1(train_audio_dataset)
val_dataset1 = Sound_ds1(val_audio_dataset)
test_dataset1 = Sound_ds1(test_audio_dataset)

In [ ]:
batch_size = 8
train_loader1 = DataLoader(train_dataset1,batch_size=batch_size,shuffle=True)
val_loader1 = DataLoader(val_dataset1,batch_size=batch_size,shuffle=False)
test_loader1 = DataLoader(test_dataset1,batch_size=batch_size,shuffle=False)

Creating the classifer on top of Whisper model

In [ ]:
class extend_model(nn.Module):
  def __init__(self,model):
    super(extend_model,self).__init__()
    self.model = model
    self.classifier = nn.Sequential(
        nn.Linear(self.model.config.hidden_size,2048),
        nn.ReLU(),
        nn.Linear(2048,1024),
        nn.ReLU(),
        nn.Linear(1024,512),
        nn.ReLU(),
        nn.Linear(512,10)
    )

  def forward(self,x,x_ids):
    outputs = self.model(x,decoder_input_ids=x_ids)
    pool_out = outputs['last_hidden_state'][:,0,:]
    out = self.classifier(pool_out)

    return out


In [ ]:
model1 = extend_model(model)
model1.to(device)

extend_model(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 512, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(512, 512, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 512)
      (layers): ModuleList(
        (0-5): 6 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=False)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): Lay

In [ ]:
lr = 2e-5
optimizer1 = optim.AdamW(model1.parameters(),lr=lr)
epochs = 10
loss_f = nn.CrossEntropyLoss()

Training

In [ ]:
train_loss1 = []
vali_loss1 = []

def train(model,loss_f,train_loader,val_loader,optimizer,train_loss,val_loss):
  for epoch in range(10):
    model.train()
    batch_losses = []
    for i,data in enumerate(train_loader):
      optimizer.zero_grad()
      x = data[0].squeeze().to(device)
      x_ids = data[1].squeeze().to(device)
      y = data[2].view(-1).to(device)

      y_pred = model(x,x_ids)
      loss = loss_f(y_pred,y)
      loss.backward()
      batch_losses.append(loss.item())
      optimizer.step()

    train_loss.append(batch_losses)
    print(f'Epoch - {epoch} Train-Loss : {np.mean(train_loss[-1])}')

    model.eval()

    val_batch_losses = []

    with torch.no_grad():
      for i,data in enumerate(val_loader):
        x = data[0].squeeze().to(device)
        x_ids = data[1].squeeze().to(device)
        y = data[2].view(-1).to(device)

        y_pred = model(x,x_ids)
        val_loss1 = loss_f(y_pred,y)
        val_batch_losses.append(val_loss1.item())

    val_loss.append(val_batch_losses)
    print(f'Epoch - {epoch} Validation-Loss : {np.mean(val_loss[-1])}')


In [ ]:
train(model1,loss_f,train_loader1,val_loader1,optimizer1,train_loss1,vali_loss1)

Epoch - 0 Train-Loss : 1.4368999816753245
Epoch - 0 Validation-Loss : 0.9717940807342529
Epoch - 1 Train-Loss : 0.7824049732751317
Epoch - 1 Validation-Loss : 1.2307195127010346
Epoch - 2 Train-Loss : 0.5078715595796153
Epoch - 2 Validation-Loss : 0.6457733601331711
Epoch - 3 Train-Loss : 0.4056312574142659
Epoch - 3 Validation-Loss : 0.7256020794312159
Epoch - 4 Train-Loss : 0.33153637462054136
Epoch - 4 Validation-Loss : 0.562277719626824
Epoch - 5 Train-Loss : 0.24863948922052428
Epoch - 5 Validation-Loss : 1.1424733142058054
Epoch - 6 Train-Loss : 0.21445703542342892
Epoch - 6 Validation-Loss : 0.7570129096508026
Epoch - 7 Train-Loss : 0.19540631783160348
Epoch - 7 Validation-Loss : 0.632824195548892
Epoch - 8 Train-Loss : 0.1757864401830981
Epoch - 8 Validation-Loss : 0.7117993885030349
Epoch - 9 Train-Loss : 0.13935226732864975
Epoch - 9 Validation-Loss : 0.5672649512688319


Evaluating

In [ ]:
model1.eval()

predictions = []
true_labels = []
correct_pred = 0
total_pred = 0
with torch.no_grad():
  for i,data in enumerate(test_loader1):
    if(i%100==0):
      print(i)
    x = data[0].squeeze().to(device)
    x_ids = data[1].squeeze().to(device)
    y = data[2].view(-1).to(device)

    # print(x.shape)
    # print(y.shape)


    outputs = model1(x,x_ids)
    _,y_pred = torch.max(outputs,1)
    # print(y_pred,y)
    predictions.extend(y_pred.cpu().numpy())
    true_labels.extend(y.cpu().numpy())
    # correct_pred +=(y_pred==y).sum().item()
    # total_pred += y_pred.shape[0]



0


In [ ]:
accuracy = accuracy_score(predictions,true_labels)
print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 84.33%


In [ ]:
precision = precision_score(predictions,true_labels,average = "weighted")
print(f"Precision: {precision:.2f}")

Precision: 0.86


In [ ]:
recall = recall_score(predictions,true_labels,average = "weighted")
print(f"Recall: {recall:.2f}")

Recall: 0.84


In [ ]:
f1score = f1_score(predictions,true_labels,average = "weighted")
print(f"F1_score: {f1score:.2f}")

F1_score: 0.85


In [ ]:
print(classification_report(predictions,true_labels,target_names=sorted(classes)))

                  precision    recall  f1-score   support

 air_conditioner       0.70      0.78      0.74        27
        car_horn       0.80      0.80      0.80        20
children_playing       0.94      0.78      0.85        40
        dog_bark       0.89      0.96      0.93        26
        drilling       0.85      0.93      0.89        30
   engine_idling       0.81      0.83      0.82        36
        gun_shot       1.00      1.00      1.00         7
      jackhammer       0.97      0.77      0.86        47
           siren       0.94      0.86      0.90        37
    street_music       0.66      0.90      0.76        30

        accuracy                           0.84       300
       macro avg       0.86      0.86      0.85       300
    weighted avg       0.86      0.84      0.85       300

